In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import shutil

In [ ]:
from sklearn import svm

In [ ]:
import pickle

In [ ]:
# Function to Extract features from the images
def image_feature(direc):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = [];
    img_name = [];
    if os.path.isdir(direc):
      for files in tqdm(os.listdir(direc)):
          fname=direc+'/'+files
          img=image.load_img(fname,target_size=(224,224))
          x = img_to_array(img)
          x=np.expand_dims(x,axis=0)
          x=preprocess_input(x)
          feat=model.predict(x)
          feat=feat.flatten()
          features.append(feat)
          img_name.append(files)
      return features,img_name
    else:
      files=direc.split("/")[-1]
      fname=direc
      img=image.load_img(fname,target_size=(224,224))
      x = img_to_array(img)
      x=np.expand_dims(x,axis=0)
      x=preprocess_input(x)
      feat=model.predict(x)
      feat=feat.flatten()
      features.append(feat)
      img_name.append(files)
      return features,img_name

#### Training

In [ ]:
dataset_path="/content/Docs"
n=len(dataset_path)

In [ ]:
labels = [label for label in os.listdir(dataset_path)]
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id

{'.ipynb_checkpoints': 3, 'InvoiceTable': 0, 'Mixed': 1, 'Text': 2}

In [ ]:
images = []
labels = []

for label_folder, _, file_names in os.walk(dataset_path):
  if label_folder != dataset_path:
    label = label_folder[n:]
    for _, _, image_names in os.walk(label_folder):
      relative_image_names = []
      for image_file in image_names:
        relative_image_names.append(dataset_path + "/" + label + "/" + image_file)
      images.extend(relative_image_names)
      labels.extend([label] * len (relative_image_names)) 

data = pd.DataFrame.from_dict({'image_path': images, 'label': labels})
data.head()

In [ ]:
train_y=list(data["label"])


In [ ]:
data.head()

,image_path,label
0,/content/Docs//InvoiceTable/1108-receipt.jpg,/InvoiceTable
1,/content/Docs//InvoiceTable/1100-receipt.jpg,/InvoiceTable
2,/content/Docs//InvoiceTable/tables11.jpg,/InvoiceTable
3,/content/Docs//InvoiceTable/1153-receipt.jpg,/InvoiceTable
4,/content/Docs//InvoiceTable/tables9.png,/InvoiceTable


In [ ]:
train_x=[]
for i in tqdm(data.image_path):
  img_features,img_name=image_feature(str(i))
  train_x.append(img_features)


100%|██████████| 209/209 [14:11<00:00,  4.07s/it]


In [ ]:
train_x_1=train_x

In [ ]:
train_x_2=np.array(train_x_1)

In [ ]:
train_x_2.shape

(209, 1, 51200)

In [ ]:
nsamples, nx, ny = train_x_2.shape
train_dataset = train_x_2.reshape((nsamples,nx*ny))

In [ ]:
clf_svm_wv = svm.SVC(kernel='rbf')
clf_svm_wv.fit(train_dataset, train_y)

SVC()

list

#### Testing

In [ ]:
path="/content/Test_3_tables"
test_x=[]
img_name=[]
if os.path.isdir(direc):
  for files in tqdm(os.listdir(direc)):
  img_name.append(files)
  fname=direc+'/'+files
  img_features,im_name=image_feature(str(fname))
  test_x.append(img_features)
          
else:
  files=direc.split("/")[-1]
  fname=direc
  img_name.append(files)
  img_features,im_name=image_feature(str(fname))
  test_x.append(img_features)

In [ ]:
os.listdir("/content/Train_3/1080-receipt.jpg")

NotADirectoryError: ignored

In [ ]:
img_name=[]
for i in tqdm(os.listdir(path)):
  img_name.append(i)

100%|██████████| 75/75 [00:00<00:00, 90472.48it/s]


In [ ]:
img_name

In [ ]:
test_x_1=np.array(test_x)
nsamples, nx, ny = test_x_1.shape
test_dataset = test_x_1.reshape((nsamples,nx*ny))

In [ ]:
test_y=clf_svm_wv.predict(test_dataset)

In [ ]:
len(test_y)

75

In [ ]:
clf_svm_wv.predict(test_dataset)

In [ ]:
Test_df = pd.DataFrame(img_name,columns=['image'])

In [ ]:
Test_df

,image
0,download (5).png
1,1075-receipt.jpg
2,tables15.jpg
3,1078-receipt.jpg
4,0148_271.png
...,...
70,0110_099.png
71,tables12.jpg
72,1740_002.png
73,download (2).jpg


In [ ]:
Test_df["label"]=test_y

In [ ]:
Test_df

In [ ]:
pickle.dump(clf_svm_wv, open("save_svm_feature.pkl", "wb"))